In [1]:
# import packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)

In [2]:
# get datasets
dataset = pd.read_excel('../data/us_state_2019.xlsx', header=0)
dataset1 = pd.read_csv('../data/cbp20cd.csv', encoding = 'latin-1', sep = ";")

In [ ]:
# get columns for employee and payroll totals
# get employee and payroll total
df1 = pd.DataFrame(dataset1)
# should remove ---
df1['2017 NAICS Code'] = df1['2017 NAICS Code'].str.replace("-", "")
# should give 0 instead of NA
df1['2017 NAICS Code'] = df1['2017 NAICS Code'].fillna(0)
# take only certain rows
df_new = pd.DataFrame(df1[:9093])
df_new[" Number of Establishments "]
df_new["State FIPS"] = df_new['State FIPS'].astype(int)
# should remove ---
df_new[" Number of Establishments "] = df_new[" Number of Establishments "].str.replace(" ", "")
df_new[" Number of Establishments "] = df_new[" Number of Establishments "].astype(int)
df_new[" Employment "] = df_new[" Employment "].str.replace(" ", "")
df_new[" Employment "] = df_new[" Employment "].astype(int)
df_new[" Annual Payroll ($1,000) "] = df_new[" Annual Payroll ($1,000) "].str.replace(" ", "")
df_new[" Annual Payroll ($1,000) "] = df_new[" Annual Payroll ($1,000) "].astype(int)
# Aggregate all values
merge = df_new[["State FIPS", " Number of Establishments ", '2017 NAICS Code', " Employment ",
                " Annual Payroll ($1,000) ", 'NAICS Description', "State"]].groupby(['State FIPS',
                                                       '2017 NAICS Code', 'NAICS Description','State']).agg('sum')
# remove grouping to get separate columns
merge = merge.reset_index()
# get all 50 states lines
df11=merge.pivot(index='State', columns='NAICS Description', 
                 values = [" Employment ", " Number of Establishments ", " Annual Payroll ($1,000) "])
df11 = df11.reset_index()
# rename totals

# get totals
empTOT = df11[' Annual Payroll ($1,000) ', 'Total for all sectors']
payTOT = df11[' Employment ', 'Total for all sectors']
#combine with state
totals = pd.concat([df11['State'],empTOT,payTOT], axis=1)
totals.rename(columns = {(' Annual Payroll ($1,000) ', 'Total for all sectors'): 'PayTOT',
                                  (' Employment ', 'Total for all sectors'): 'EmpTOT'},inplace = True)
# drop Hawaii and Puerto Rico
totals = totals[totals.State != "Hawaii"]
totals = totals[totals.State != "Puerto Rico"]
# reset and drop index
totals = totals.reset_index()
totals = totals.drop(columns = ['index'])
totals

,State,PayTOT,EmpTOT
0,Alabama,7.851187e+07,1703952.0
1,Alaska,1.633103e+07,266063.0
2,Arizona,1.277071e+08,2510877.0
3,Arkansas,4.651172e+07,1026484.0
4,California,1.064073e+09,14812024.0
5,Colorado,1.350553e+08,2366541.0
6,Connecticut,9.794615e+07,1520091.0
7,Delaware,2.511497e+07,422044.0
8,District of Columbia,4.585003e+07,543174.0
9,Florida,3.564499e+08,7530942.0


In [ ]:
# drop and change columns
dataset.rename(columns = {'Annual Payroll':'Annual_Payroll'}, inplace = True)

#drop --
data = dataset[dataset["SectorCode"].str.contains("--") == False]

#keep only total inside
data=data[data["Enterprise Size"].str.contains("01: Total")==True]

#drop 01:total
data=data.drop(['Enterprise Size'], axis = 1)
#columns
columns=data.columns.values.tolist()

In [ ]:
# types and create sector

#types 
types=data.dtypes


###CREATE SECTORS 

agriculture = data[(data["SectorCode"] == "11") ]


agriculture=agriculture.drop(['Sector'], axis = 1)
agriculture.rename(columns = {'SectorCode': 'agriculture'},inplace=True)

agriculture["agriculture"] = pd.to_numeric(agriculture["agriculture"])

agriculture.rename(columns = {'Annual_Payroll':'Annual_Payroll_agriculture'}, inplace = True)
agriculture.rename(columns = {'Establishments':'Establishments_agriculture'}, inplace = True)
agriculture.rename(columns = {'Employment':'Employment_agriculture'}, inplace = True)

columns_agri=agriculture.columns.values.tolist()
types_agri=agriculture.dtypes

chemicals =data[(data["SectorCode"] == "325") ]


chemicals=chemicals.drop(['Sector'], axis = 1)


chemicals.rename(columns = {'SectorCode':'Chemicals'}, inplace = True)

chemicals["Chemicals"] = pd.to_numeric(chemicals["Chemicals"])

chemicals.rename(columns = {'Annual_Payroll':'Annual_Payroll_chemicals'}, inplace = True)
chemicals.rename(columns = {'Establishments':'Establishments_chemicals'}, inplace = True)
chemicals.rename(columns = {'Employment':'Employment_chemicals'}, inplace = True)

types_chem=chemicals.dtypes




construction=data[(data["SectorCode"] == "23")]

construction=construction.drop(['Sector'], axis = 1)


construction.rename(columns = {'SectorCode':'Construction'},inplace=True)

construction["Construction"] = pd.to_numeric(construction["Construction"])

construction.rename(columns = {'Annual_Payroll':'Annual_Payroll_construction'}, inplace = True)
construction.rename(columns = {'Establishments':'Establishments_construction'}, inplace = True)
construction.rename(columns = {'Employment':'Employment_construction'}, inplace = True)

types_constr=construction.dtypes




''' 2 categoriesin 'electrical' '''
electrical_power=data[(data["SectorCode"] == "2211")|(data["SectorCode"] == "2212")]



electrical_power.rename(columns = {'SectorCode':'electrical'}, inplace = True)


electrical_power["electrical"] = pd.to_numeric(electrical_power["electrical"])

electrical_power.rename(columns = {'Annual_Payroll':'Annual_Payroll_electrical'}, inplace = True)
electrical_power.rename(columns = {'Establishments':'Establishments_electrical'}, inplace = True)
electrical_power.rename(columns = {'Employment':'Employment_electrical'}, inplace = True)

electrical_power=electrical_power.groupby('StateCode').sum()

types_electr=electrical_power.dtypes


'''1 category in 'food' '''
food_and_beverage=data[(data["SectorCode"] == "3121")|(data["SectorCode"] == "311")]


#rename the columns
food_and_beverage.rename(columns = {'SectorCode':'food'}, inplace = True)

food_and_beverage["food"] = pd.to_numeric(food_and_beverage["food"])

food_and_beverage.rename(columns = {'Annual_Payroll':'Annual_Payroll_food'}, inplace = True)
food_and_beverage.rename(columns = {'Establishments':'Establishments_food'}, inplace = True)
food_and_beverage.rename(columns = {'Employment':'Employment_food'}, inplace = True)

food_and_beverage=food_and_beverage.groupby('StateCode').sum()

types_food=food_and_beverage.dtypes


''' 5 categories for 'mining' '''
mining=data[(data["SectorCode"] == "212")|(data["SectorCode"] == "213") | (data["SectorCode"] == "213114")|(data["SectorCode"] == "213113")|(data["SectorCode"] == "213115")]


#rename the columns

mining.rename(columns = {'SectorCode':'mining'}, inplace = True)

mining["mining"] = pd.to_numeric(mining["mining"])

mining.rename(columns = {'Annual_Payroll ':'Annual_Payroll_mining'}, inplace = True)
mining.rename(columns = {'Establishments':'Establishments_mining'}, inplace = True)
mining.rename(columns = {'Employment':'Employment_mining'}, inplace = True)

mining=mining.groupby('StateCode').sum()


''' 2 categories for 'oil_and_gas' '''
oil_and_gas=data[(data["SectorCode"] == "211")|(data["SectorCode"] == "324") | (data["SectorCode"] == "213112")]

#rename the columns

oil_and_gas.rename(columns = {'SectorCode':'oil_and_gas'}, inplace = True)

oil_and_gas["oil_and_gas"] = pd.to_numeric(oil_and_gas["oil_and_gas"])

oil_and_gas.rename(columns = {'Annual_Payroll':'Annual_Payroll_oil_and_gas'}, inplace = True)
oil_and_gas.rename(columns = {'Establishments':'Establishments_oil_and_gas'}, inplace = True)
oil_and_gas.rename(columns = {'Employment':'Employment_oil_and_gas'}, inplace = True)

oil_and_gas=oil_and_gas.groupby('StateCode').sum()



''' 2 categories for 'semiconductor' '''
semiconductor=data[(data["SectorCode"] == "334")|(data["SectorCode"] == "335") | (data["SectorCode"] == "336")]

#rename the columns

semiconductor.rename(columns = {'SectorCode':'semiconductor'}, inplace = True)

semiconductor["semiconductor"] = pd.to_numeric(semiconductor["semiconductor"])

semiconductor.rename(columns = {'Annual_Payroll':'Annual_Payroll_semiconductor'}, inplace = True)
semiconductor.rename(columns = {'Establishments':'Establishments_semiconductor'}, inplace = True)
semiconductor.rename(columns = {'Employment':'Employment_semiconductor'}, inplace = True)

semiconductor=semiconductor.groupby('StateCode').sum()




'''5 different values for 'textile' '''
textile=data[(data["SectorCode"] == "313")|(data["SectorCode"] == "314") | (data["SectorCode"] == "315") |(data["SectorCode"] == "316")]

#rename the columns

textile.rename(columns = {'SectorCode':'textile'}, inplace = True)

textile["textile"] = pd.to_numeric(textile["textile"])

textile.rename(columns = {'Annual_Payroll':'Annual_Payroll_textile'}, inplace = True)
textile.rename(columns = {'Establishments':'Establishments_textile'}, inplace = True)
textile.rename(columns = {'Employment':'Employment_textile'}, inplace = True)

textile=textile.groupby('StateCode').sum()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [ ]:
#####################FINAL DATASET#################

m1=pd.merge(agriculture, chemicals, on ='StateCode')
m2=pd.merge(construction, electrical_power, on ='StateCode')
m3=pd.merge(food_and_beverage, mining, on ='StateCode')
m4=pd.merge(oil_and_gas, semiconductor, on ='StateCode')

m5=pd.merge(m1, m2, how='outer', on ='StateCode') 

m6=pd.merge(m5, m3, how='outer', on ='StateCode')

m7=pd.merge(m6, m4, how='outer', on ='StateCode')

final=pd.merge(m7, textile, how='outer', on ='StateCode')



#drop sector columns
final=final.drop(['agriculture'], axis = 1)
final=final.drop(['Chemicals'], axis = 1)
final=final.drop(['Construction'], axis = 1)
final=final.drop(['electrical'], axis = 1)
final=final.drop(['food'], axis = 1)
final=final.drop(['mining'], axis = 1)
final=final.drop(['oil_and_gas'],axis=1)
final=final.drop(['semiconductor'], axis = 1)
final=final.drop(['textile'], axis = 1)

#fill Nas with 0
final=final.fillna(0)


##remove 1st row which is united states
N =1
final= final.iloc[N: , :]

##remove hawaii
f1=final.iloc[0:10, :]
f2=final.iloc[11:51, :]

final=pd.concat([f1,f2]).reset_index(drop=True)

##drop state_x , state_y  and enterprise Size
final=final.drop(['State _x'], axis = 1)
final=final.drop(['State _y'], axis = 1)

for col in final.columns:
  print(col)

#final=final.drop(['Enterprise Size_x'], axis = 1)
#final=final.drop(['Enterprise Size'], axis = 1)
#final=final.drop(['Enterprise Size_y'], axis = 1)

final.rename(columns = {'Annual_Payroll':'Annual_Payroll_mining'}, inplace = True)

# Rename State column
final.rename(columns = {'State ':'State'}, inplace = True)

# Put state first
cols = list(final.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('State')) #Remove b from list
final = final[['State']+cols] #Create new dataframe with columns in the order you want

final=pd.merge(final,totals, how='outer', on ='State')

#final.drop('Unnamed: 0', axis=1, inplace=True)


####devide the annual_payroll anf the employment with the totals of the states
#nine times (one for each industry)

#agriculture
final[final.columns[3:4]] =final[final.columns[3:4]].div(final['EmpTOT'], axis=0).round(4)
final[final.columns[4:5]] =final[final.columns[4:5]].div(final['PayTOT'], axis=0).round(4)

#chemicals
final[final.columns[6:7]] =final[final.columns[6:7]].div(final['EmpTOT'], axis=0).round(4)
final[final.columns[7:8]] =final[final.columns[7:8]].div(final['PayTOT'], axis=0).round(4)

#construstions
final[final.columns[9:10]] =final[final.columns[9:10]].div(final['EmpTOT'], axis=0).round(4)
final[final.columns[10:11]] =final[final.columns[10:11]].div(final['PayTOT'], axis=0).round(4)

#electrical
final[final.columns[12:13]] =final[final.columns[12:13]].div(final['EmpTOT'], axis=0).round(4)
final[final.columns[13:14]] =final[final.columns[13:14]].div(final['PayTOT'], axis=0).round(4)

#food
final[final.columns[15:16]] =final[final.columns[15:16]].div(final['EmpTOT'], axis=0).round(4)
final[final.columns[16:17]] =final[final.columns[16:17]].div(final['PayTOT'], axis=0).round(4)

#mining
final[final.columns[18:19]] =final[final.columns[18:19]].div(final['EmpTOT'], axis=0).round(4)
final[final.columns[19:20]] =final[final.columns[19:20]].div(final['PayTOT'], axis=0).round(4)

#oil and gas
final[final.columns[21:22]] =final[final.columns[21:22]].div(final['EmpTOT'], axis=0).round(4)
final[final.columns[22:23]] =final[final.columns[22:23]].div(final['PayTOT'], axis=0).round(4)

#semiconductor
final[final.columns[24:25]] =final[final.columns[24:25]].div(final['EmpTOT'], axis=0).round(4)
final[final.columns[25:26]] =final[final.columns[25:26]].div(final['PayTOT'], axis=0).round(4)

#textile
final[final.columns[27:28]] =final[final.columns[27:28]].div(final['EmpTOT'], axis=0).round(4)
final[final.columns[28:29]] =final[final.columns[28:29]].div(final['PayTOT'], axis=0).round(4)


#columns
columns_final=final.columns.values.tolist()

# output final df
final.to_csv('../data/us_state_digit_2019.csv')


StateCode
Establishments_agriculture
Employment_agriculture
Annual_Payroll_agriculture
Establishments_chemicals
Employment_chemicals
Annual_Payroll_chemicals
State 
Establishments_construction
Employment_construction
Annual_Payroll_construction
Establishments_electrical
Employment_electrical
Annual_Payroll_electrical
Establishments_food
Employment_food
Annual_Payroll_food
Establishments_mining
Employment_mining
Annual_Payroll
Establishments_oil_and_gas
Employment_oil_and_gas
Annual_Payroll_oil_and_gas
Establishments_semiconductor
Employment_semiconductor
Annual_Payroll_semiconductor
Establishments_textile
Employment_textile
Annual_Payroll_textile
